In [1]:
!pip install requests
!pip install ftplib


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement ftplib (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for ftplib


In [46]:
### import library
import datetime
import time
from ftplib import FTP
import pandas as pd
import io
import os

In [49]:
username = 'anonymous'
password = 'useremailaddress' 
## Set start and end time, to filter last 12 months files
today = datetime.datetime.now()

starttime  = today.replace(day=1) - datetime.timedelta(days=1096) ### last 36 month
endtime = today.replace(day=1) - datetime.timedelta(days=1) ### last month

# Generate list of YYYY-MM dates
date_list = []
current_datetime = starttime
while current_datetime <= endtime:
    date_list.append(current_datetime.strftime('%Y%m'))
    current_datetime += datetime.timedelta(days=31)  # Add one month

print("List of YYYY-MM dates between start and end time:")
print(date_list)
# Connect to the FTP server to get list of location in NSW
def get_observation_location(ftp_server, ftp_directory ):
    with FTP(ftp_server) as ftp:
        ftp.login(username, password)
        ftp.cwd(ftp_directory)
        
        # Get list of directories in the current directory
        observe_locations_list = ftp.nlst()
        return observe_locations_list
# Function to download files from FTP server
def download_files_from_ftp(ftp_server, ftp_directory, local_directory, observe_location, date_list):
      with FTP(ftp_server) as ftp:

        ftp.login(username, password)
        ftp_directory_location = ftp_directory +  observe_location + '/'
        ftp.cwd(ftp_directory_location)
        filenames = [f"{observe_location}-{date}" for date in date_list ]
                    
        for filename in filenames:
            remote_filepath = f"{filename}.csv"
            local_filepath = f"{local_directory}/{filename}.csv"

            try:
                with open(local_filepath, "wb") as local_file:
                    ftp.retrbinary(f"RETR {remote_filepath}", local_file.write)   
                    print(f"File '{remote_filepath}' downloaded to '{local_filepath}'")

            except Exception as e:
                print(f"Error downloading file '{remote_filepath}': {e}")

List of YYYY-MM dates between start and end time:
['202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304', '202305', '202306', '202307', '202308', '202309', '202310', '202311', '202312', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411', '202412', '202501', '202502', '202503', '202504']


In [56]:
import datetime
import time
from ftplib import FTP, error_perm
import os

username = 'anonymous'
password = 'useremailaddress'

# Past 36 month of data
today = datetime.datetime.now()
starttime  = today.replace(day=1) - datetime.timedelta(days=1096)
endtime    = today.replace(day=1) - datetime.timedelta(days=1)

# list of time download
date_list = []
current_datetime = starttime
while current_datetime <= endtime:
    date_list.append(current_datetime.strftime('%Y%m'))
    current_datetime += datetime.timedelta(days=31)
print("Date：", date_list)

def get_observation_locations(ftp_server, ftp_directory):
    with FTP(ftp_server) as ftp:
        ftp.login(username, password)
        ftp.cwd(ftp_directory)
        return ftp.nlst()

def download_files_from_ftp(ftp_server, ftp_directory, local_directory, observe_location, date_list):
    with FTP(ftp_server) as ftp:
        ftp.login(username, password)
        ftp_directory_location = ftp_directory + observe_location + '/'
        # if can not enter then skip
        try:
            ftp.cwd(ftp_directory_location)
        except error_perm as e:
            print(f"[Skip] Fail to enter：{observe_location} -> {e}")
            return

        # download file for every months
        for date in date_list:
            filename = f"{observe_location}-{date}.csv"
            remote_filepath = filename
            local_filepath = os.path.join(local_directory, filename)

            try:
                with open(local_filepath, "wb") as local_file:
                    ftp.retrbinary(f"RETR {remote_filepath}", local_file.write)
                print(f"✔ Downloaded：{remote_filepath} → {local_filepath}")
            except Exception as e:
                # dont stop download just skip
                print(f"[Skip] Download Fail：{remote_filepath} -> {e}")
                continue

# main
ftp_server     = "ftp.bom.gov.au"
ftp_directory  = "/anon/gen/clim_data/IDCKWCDEA0/tables/nsw/"
local_directory = "E:/study/meteoroData/nsw"

# 1. get all data get_observation_locations
observe_locations = get_observation_locations(ftp_server, ftp_directory)

# 2. since already download A-C, skip and starting from D 
for loc in observe_locations:
    if loc[0].upper() < 'P':
        print(f"[Skip] No need to download：{loc}")
        continue

    # 3. download
    download_files_from_ftp(ftp_server, ftp_directory, local_directory, loc, date_list)
    time.sleep(20)
    

Date： ['202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304', '202305', '202306', '202307', '202308', '202309', '202310', '202311', '202312', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411', '202412', '202501', '202502', '202503', '202504']
[Skip] No need to download：albion_park_(shellharbour_airport)
[Skip] No need to download：albion_park_(wollongong_airport)
[Skip] No need to download：albury_airport
[Skip] No need to download：armidale_airport
[Skip] No need to download：badgerys_creek
[Skip] No need to download：ballina_airport
[Skip] No need to download：bankstown_airport
[Skip] No need to download：bathurst_airport
[Skip] No need to download：bega
[Skip] No need to download：bellambi
[Skip] No need to download：bombala
[Skip] No need to download：borrona_downs
[Skip] No need to download：bourke_airport
[Skip] No need to download：braidwood_racecourse
[Skip] No need to

✔ Downloaded：paterson_(tocal)-202209.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202209.csv
✔ Downloaded：paterson_(tocal)-202210.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202210.csv
✔ Downloaded：paterson_(tocal)-202211.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202211.csv
✔ Downloaded：paterson_(tocal)-202212.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202212.csv
✔ Downloaded：paterson_(tocal)-202301.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202301.csv
✔ Downloaded：paterson_(tocal)-202302.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202302.csv
✔ Downloaded：paterson_(tocal)-202303.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202303.csv
✔ Downloaded：paterson_(tocal)-202304.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202304.csv
✔ Downloaded：paterson_(tocal)-202305.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202305.csv
✔ Downloaded：paterson_(tocal)-202306.csv → E:/study/meteoroData/nsw\paterson_(tocal)-202306.csv
✔ Downloaded：paterson_(tocal)-202307.csv

✔ Downloaded：perisher_valley-202403.csv → E:/study/meteoroData/nsw\perisher_valley-202403.csv
✔ Downloaded：perisher_valley-202404.csv → E:/study/meteoroData/nsw\perisher_valley-202404.csv
✔ Downloaded：perisher_valley-202405.csv → E:/study/meteoroData/nsw\perisher_valley-202405.csv
✔ Downloaded：perisher_valley-202406.csv → E:/study/meteoroData/nsw\perisher_valley-202406.csv
✔ Downloaded：perisher_valley-202407.csv → E:/study/meteoroData/nsw\perisher_valley-202407.csv
✔ Downloaded：perisher_valley-202408.csv → E:/study/meteoroData/nsw\perisher_valley-202408.csv
✔ Downloaded：perisher_valley-202409.csv → E:/study/meteoroData/nsw\perisher_valley-202409.csv
✔ Downloaded：perisher_valley-202410.csv → E:/study/meteoroData/nsw\perisher_valley-202410.csv
✔ Downloaded：perisher_valley-202411.csv → E:/study/meteoroData/nsw\perisher_valley-202411.csv
✔ Downloaded：perisher_valley-202412.csv → E:/study/meteoroData/nsw\perisher_valley-202412.csv
✔ Downloaded：perisher_valley-202501.csv → E:/study/meteoroDa

[Skip] Download Fail：port_macquarie_airport_(comparison)-202301.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202302.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202303.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202304.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202305.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202306.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202307.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202308.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202309.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airport_(comparison)-202310.csv -> 550 Failed to open file.
[Skip] Download Fail：port_macquarie_airp

✔ Downloaded：richmond_raaf-202307.csv → E:/study/meteoroData/nsw\richmond_raaf-202307.csv
✔ Downloaded：richmond_raaf-202308.csv → E:/study/meteoroData/nsw\richmond_raaf-202308.csv
✔ Downloaded：richmond_raaf-202309.csv → E:/study/meteoroData/nsw\richmond_raaf-202309.csv
✔ Downloaded：richmond_raaf-202310.csv → E:/study/meteoroData/nsw\richmond_raaf-202310.csv
✔ Downloaded：richmond_raaf-202311.csv → E:/study/meteoroData/nsw\richmond_raaf-202311.csv
✔ Downloaded：richmond_raaf-202312.csv → E:/study/meteoroData/nsw\richmond_raaf-202312.csv
✔ Downloaded：richmond_raaf-202401.csv → E:/study/meteoroData/nsw\richmond_raaf-202401.csv
✔ Downloaded：richmond_raaf-202402.csv → E:/study/meteoroData/nsw\richmond_raaf-202402.csv
✔ Downloaded：richmond_raaf-202403.csv → E:/study/meteoroData/nsw\richmond_raaf-202403.csv
✔ Downloaded：richmond_raaf-202404.csv → E:/study/meteoroData/nsw\richmond_raaf-202404.csv
✔ Downloaded：richmond_raaf-202405.csv → E:/study/meteoroData/nsw\richmond_raaf-202405.csv
✔ Download

...

✔ Downloaded：walgett_airport-202311.csv → E:/study/meteoroData/nsw\walgett_airport-202311.csv
✔ Downloaded：walgett_airport-202312.csv → E:/study/meteoroData/nsw\walgett_airport-202312.csv
✔ Downloaded：walgett_airport-202401.csv → E:/study/meteoroData/nsw\walgett_airport-202401.csv
✔ Downloaded：walgett_airport-202402.csv → E:/study/meteoroData/nsw\walgett_airport-202402.csv
✔ Downloaded：walgett_airport-202403.csv → E:/study/meteoroData/nsw\walgett_airport-202403.csv
✔ Downloaded：walgett_airport-202404.csv → E:/study/meteoroData/nsw\walgett_airport-202404.csv
✔ Downloaded：walgett_airport-202405.csv → E:/study/meteoroData/nsw\walgett_airport-202405.csv
✔ Downloaded：walgett_airport-202406.csv → E:/study/meteoroData/nsw\walgett_airport-202406.csv
✔ Downloaded：walgett_airport-202407.csv → E:/study/meteoroData/nsw\walgett_airport-202407.csv
✔ Downloaded：walgett_airport-202408.csv → E:/study/meteoroData/nsw\walgett_airport-202408.csv
✔ Downloaded：walgett_airport-202409.csv → E:/study/meteoroDa

[Skip] Download Fail：wanaaring_(delta)-202501.csv -> 550 Failed to open file.
[Skip] Download Fail：wanaaring_(delta)-202502.csv -> 550 Failed to open file.
[Skip] Download Fail：wanaaring_(delta)-202503.csv -> 550 Failed to open file.
[Skip] Download Fail：wanaaring_(delta)-202504.csv -> 550 Failed to open file.
✔ Downloaded：west_wyalong_airport-202205.csv → E:/study/meteoroData/nsw\west_wyalong_airport-202205.csv
✔ Downloaded：west_wyalong_airport-202206.csv → E:/study/meteoroData/nsw\west_wyalong_airport-202206.csv
✔ Downloaded：west_wyalong_airport-202207.csv → E:/study/meteoroData/nsw\west_wyalong_airport-202207.csv
✔ Downloaded：west_wyalong_airport-202208.csv → E:/study/meteoroData/nsw\west_wyalong_airport-202208.csv
✔ Downloaded：west_wyalong_airport-202209.csv → E:/study/meteoroData/nsw\west_wyalong_airport-202209.csv
✔ Downloaded：west_wyalong_airport-202210.csv → E:/study/meteoroData/nsw\west_wyalong_airport-202210.csv
✔ Downloaded：west_wyalong_airport-202211.csv → E:/study/meteoroD

✔ Downloaded：wilcannia_aerodrome-202303.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202303.csv
✔ Downloaded：wilcannia_aerodrome-202304.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202304.csv
✔ Downloaded：wilcannia_aerodrome-202305.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202305.csv
✔ Downloaded：wilcannia_aerodrome-202306.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202306.csv
✔ Downloaded：wilcannia_aerodrome-202307.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202307.csv
✔ Downloaded：wilcannia_aerodrome-202308.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202308.csv
✔ Downloaded：wilcannia_aerodrome-202309.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202309.csv
✔ Downloaded：wilcannia_aerodrome-202310.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202310.csv
✔ Downloaded：wilcannia_aerodrome-202311.csv → E:/study/meteoroData/nsw\wilcannia_aerodrome-202311.csv
✔ Downloaded：wilcannia_aerodrome-202312.csv → E:/study/meteoroData/nsw\wilcannia_a

✔ Downloaded：yamba_pilot_station-202403.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202403.csv
✔ Downloaded：yamba_pilot_station-202404.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202404.csv
✔ Downloaded：yamba_pilot_station-202405.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202405.csv
✔ Downloaded：yamba_pilot_station-202406.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202406.csv
✔ Downloaded：yamba_pilot_station-202407.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202407.csv
✔ Downloaded：yamba_pilot_station-202408.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202408.csv
✔ Downloaded：yamba_pilot_station-202409.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202409.csv
✔ Downloaded：yamba_pilot_station-202410.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202410.csv
✔ Downloaded：yamba_pilot_station-202411.csv → E:/study/meteoroData/nsw\yamba_pilot_station-202411.csv
✔ Downloaded：yamba_pilot_station-202412.csv → E:/study/meteoroData/nsw\yamba_pilot

✔ Downloaded：young_airport-202410.csv → E:/study/meteoroData/nsw\young_airport-202410.csv
✔ Downloaded：young_airport-202411.csv → E:/study/meteoroData/nsw\young_airport-202411.csv
✔ Downloaded：young_airport-202412.csv → E:/study/meteoroData/nsw\young_airport-202412.csv
✔ Downloaded：young_airport-202501.csv → E:/study/meteoroData/nsw\young_airport-202501.csv
✔ Downloaded：young_airport-202502.csv → E:/study/meteoroData/nsw\young_airport-202502.csv
✔ Downloaded：young_airport-202503.csv → E:/study/meteoroData/nsw\young_airport-202503.csv
✔ Downloaded：young_airport-202504.csv → E:/study/meteoroData/nsw\young_airport-202504.csv


In [57]:
import re
import shutil
from pathlib import Path

# path here
base_dir = Path(r"E:/study/meteoroData/nsw")

# regular expression to read name of file
pattern = re.compile(r"^(.+)-(\d{6})\.csv$")

for csv_path in base_dir.glob("*.csv"):
    m = pattern.match(csv_path.name)
    if not m:
        # skip the one does not math any
        print(f"skip does not match format：{csv_path.name}")
        continue

    loc_name, yyyymm = m.groups()
    # 1) make file base on location name
    target_dir = base_dir / loc_name
    target_dir.mkdir(exist_ok=True)

    # 2) move csv to file
    dest_path = target_dir / csv_path.name
    shutil.move(str(csv_path), str(dest_path))
    print(f"moved: {csv_path.name} -> {loc_name}/")

moved: ivanhoe_aerodrome-202311.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202312.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202401.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202402.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202403.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202404.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202405.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202406.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202407.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202408.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202409.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202410.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202411.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202412.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202501.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202502.csv -> ivanhoe_aerodrome/
moved: ivanhoe_aerodrome-202503.csv -> ivanhoe_aerodrome/
moved: ivanhoe

moved: khancoban-202205.csv -> khancoban/
moved: khancoban-202206.csv -> khancoban/
moved: khancoban-202207.csv -> khancoban/
moved: khancoban-202208.csv -> khancoban/
moved: khancoban-202209.csv -> khancoban/
moved: khancoban-202210.csv -> khancoban/
moved: khancoban-202211.csv -> khancoban/
moved: khancoban-202212.csv -> khancoban/
moved: khancoban-202301.csv -> khancoban/
moved: khancoban-202302.csv -> khancoban/
moved: khancoban-202303.csv -> khancoban/
moved: khancoban-202304.csv -> khancoban/
moved: khancoban-202305.csv -> khancoban/
moved: khancoban-202306.csv -> khancoban/
moved: khancoban-202307.csv -> khancoban/
moved: khancoban-202308.csv -> khancoban/
moved: khancoban-202309.csv -> khancoban/
moved: khancoban-202310.csv -> khancoban/
moved: khancoban-202311.csv -> khancoban/
moved: khancoban-202312.csv -> khancoban/
moved: khancoban-202401.csv -> khancoban/
moved: khancoban-202402.csv -> khancoban/
moved: khancoban-202403.csv -> khancoban/
moved: khancoban-202404.csv -> kha

moved: mangrove_mountain-202401.csv -> mangrove_mountain/
moved: mangrove_mountain-202402.csv -> mangrove_mountain/
moved: mangrove_mountain-202403.csv -> mangrove_mountain/
moved: mangrove_mountain-202404.csv -> mangrove_mountain/
moved: mangrove_mountain-202405.csv -> mangrove_mountain/
moved: mangrove_mountain-202406.csv -> mangrove_mountain/
moved: mangrove_mountain-202407.csv -> mangrove_mountain/
moved: mangrove_mountain-202408.csv -> mangrove_mountain/
moved: mangrove_mountain-202409.csv -> mangrove_mountain/
moved: mangrove_mountain-202410.csv -> mangrove_mountain/
moved: mangrove_mountain-202411.csv -> mangrove_mountain/
moved: mangrove_mountain-202412.csv -> mangrove_mountain/
moved: mangrove_mountain-202501.csv -> mangrove_mountain/
moved: mangrove_mountain-202502.csv -> mangrove_mountain/
moved: mangrove_mountain-202503.csv -> mangrove_mountain/
moved: mangrove_mountain-202504.csv -> mangrove_mountain/
moved: marrangaroo_(defence)-202205.csv -> marrangaroo_(defence)/
moved:

moved: moree_aero-202312.csv -> moree_aero/
moved: moree_aero-202401.csv -> moree_aero/
moved: moree_aero-202402.csv -> moree_aero/
moved: moree_aero-202403.csv -> moree_aero/
moved: moree_aero-202404.csv -> moree_aero/
moved: moree_aero-202405.csv -> moree_aero/
moved: moree_aero-202406.csv -> moree_aero/
moved: moree_aero-202407.csv -> moree_aero/
moved: moree_aero-202408.csv -> moree_aero/
moved: moree_aero-202409.csv -> moree_aero/
moved: moree_aero-202410.csv -> moree_aero/
moved: moree_aero-202411.csv -> moree_aero/
moved: moree_aero-202412.csv -> moree_aero/
moved: moree_aero-202501.csv -> moree_aero/
moved: moree_aero-202502.csv -> moree_aero/
moved: moree_aero-202503.csv -> moree_aero/
moved: moree_aero-202504.csv -> moree_aero/
moved: moruya_airport-202205.csv -> moruya_airport/
moved: moruya_airport-202206.csv -> moruya_airport/
moved: moruya_airport-202207.csv -> moruya_airport/
moved: moruya_airport-202208.csv -> moruya_airport/
moved: moruya_airport-202209.csv -> moruya_a

moved: mulurulu-202405.csv -> mulurulu/
moved: mulurulu-202406.csv -> mulurulu/
moved: mulurulu-202407.csv -> mulurulu/
moved: mulurulu-202408.csv -> mulurulu/
moved: mulurulu-202409.csv -> mulurulu/
moved: mulurulu-202410.csv -> mulurulu/
moved: mulurulu-202411.csv -> mulurulu/
moved: mulurulu-202412.csv -> mulurulu/
moved: mulurulu-202501.csv -> mulurulu/
moved: mulurulu-202502.csv -> mulurulu/
moved: mulurulu-202503.csv -> mulurulu/
moved: mulurulu-202504.csv -> mulurulu/
moved: murrurundi_gap-202205.csv -> murrurundi_gap/
moved: murrurundi_gap-202206.csv -> murrurundi_gap/
moved: murrurundi_gap-202207.csv -> murrurundi_gap/
moved: murrurundi_gap-202208.csv -> murrurundi_gap/
moved: murrurundi_gap-202209.csv -> murrurundi_gap/
moved: murrurundi_gap-202210.csv -> murrurundi_gap/
moved: murrurundi_gap-202211.csv -> murrurundi_gap/
moved: murrurundi_gap-202212.csv -> murrurundi_gap/
moved: murrurundi_gap-202301.csv -> murrurundi_gap/
moved: murrurundi_gap-202302.csv -> murrurundi_gap/


...

moved: penrith_lakes-202501.csv -> penrith_lakes/
moved: penrith_lakes-202502.csv -> penrith_lakes/
moved: penrith_lakes-202503.csv -> penrith_lakes/
moved: penrith_lakes-202504.csv -> penrith_lakes/
moved: perisher_valley-202205.csv -> perisher_valley/
moved: perisher_valley-202206.csv -> perisher_valley/
moved: perisher_valley-202207.csv -> perisher_valley/
moved: perisher_valley-202208.csv -> perisher_valley/
moved: perisher_valley-202209.csv -> perisher_valley/
moved: perisher_valley-202210.csv -> perisher_valley/
moved: perisher_valley-202211.csv -> perisher_valley/
moved: perisher_valley-202212.csv -> perisher_valley/
moved: perisher_valley-202301.csv -> perisher_valley/
moved: perisher_valley-202302.csv -> perisher_valley/
moved: perisher_valley-202303.csv -> perisher_valley/
moved: perisher_valley-202304.csv -> perisher_valley/
moved: perisher_valley-202305.csv -> perisher_valley/
moved: perisher_valley-202306.csv -> perisher_valley/
moved: perisher_valley-202307.csv -> perishe

moved: scone_airport-202205.csv -> scone_airport/
moved: scone_airport-202206.csv -> scone_airport/
moved: scone_airport-202207.csv -> scone_airport/
moved: scone_airport-202208.csv -> scone_airport/
moved: scone_airport-202209.csv -> scone_airport/
moved: scone_airport-202210.csv -> scone_airport/
moved: scone_airport-202211.csv -> scone_airport/
moved: scone_airport-202212.csv -> scone_airport/
moved: scone_airport-202301.csv -> scone_airport/
moved: scone_airport-202302.csv -> scone_airport/
moved: scone_airport-202303.csv -> scone_airport/
moved: scone_airport-202304.csv -> scone_airport/
moved: scone_airport-202305.csv -> scone_airport/
moved: scone_airport-202306.csv -> scone_airport/
moved: scone_airport-202307.csv -> scone_airport/
moved: scone_airport-202308.csv -> scone_airport/
moved: scone_airport-202309.csv -> scone_airport/
moved: scone_airport-202310.csv -> scone_airport/
moved: scone_airport-202311.csv -> scone_airport/
moved: scone_airport-202312.csv -> scone_airport/


moved: sydney_olympic_park_(sydney_olympic_pk_a)-202403.csv -> sydney_olympic_park_(sydney_olympic_pk_a)/
moved: terrey_hills-202309.csv -> terrey_hills/
moved: terrey_hills-202310.csv -> terrey_hills/
moved: terrey_hills-202311.csv -> terrey_hills/
moved: terrey_hills-202312.csv -> terrey_hills/
moved: terrey_hills-202401.csv -> terrey_hills/
moved: terrey_hills-202402.csv -> terrey_hills/
moved: terrey_hills-202403.csv -> terrey_hills/
moved: terrey_hills-202404.csv -> terrey_hills/
moved: terrey_hills-202405.csv -> terrey_hills/
moved: terrey_hills-202406.csv -> terrey_hills/
moved: terrey_hills-202407.csv -> terrey_hills/
moved: terrey_hills-202408.csv -> terrey_hills/
moved: terrey_hills-202409.csv -> terrey_hills/
moved: terrey_hills-202410.csv -> terrey_hills/
moved: terrey_hills-202411.csv -> terrey_hills/
moved: terrey_hills-202412.csv -> terrey_hills/
moved: terrey_hills-202501.csv -> terrey_hills/
moved: terrey_hills-202502.csv -> terrey_hills/
moved: terrey_hills-202503.csv

moved: walgett_(brewon)-202205.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202206.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202207.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202208.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202209.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202210.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202211.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202212.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202301.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202302.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202303.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202304.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202305.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202306.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202307.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202308.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202309.csv -> walgett_(brewon)/
moved: walgett_(brewon)-202310.csv -> walgett_(b

moved: wilcannia_aerodrome-202504.csv -> wilcannia_aerodrome/
moved: williamtown_raaf-202205.csv -> williamtown_raaf/
moved: williamtown_raaf-202206.csv -> williamtown_raaf/
moved: williamtown_raaf-202207.csv -> williamtown_raaf/
moved: williamtown_raaf-202208.csv -> williamtown_raaf/
moved: williamtown_raaf-202209.csv -> williamtown_raaf/
moved: williamtown_raaf-202210.csv -> williamtown_raaf/
moved: williamtown_raaf-202211.csv -> williamtown_raaf/
moved: williamtown_raaf-202212.csv -> williamtown_raaf/
moved: williamtown_raaf-202301.csv -> williamtown_raaf/
moved: williamtown_raaf-202302.csv -> williamtown_raaf/
moved: williamtown_raaf-202303.csv -> williamtown_raaf/
moved: williamtown_raaf-202304.csv -> williamtown_raaf/
moved: williamtown_raaf-202305.csv -> williamtown_raaf/
moved: williamtown_raaf-202306.csv -> williamtown_raaf/
moved: williamtown_raaf-202307.csv -> williamtown_raaf/
moved: williamtown_raaf-202308.csv -> williamtown_raaf/
moved: williamtown_raaf-202309.csv -> will

In [58]:

states = ['nt']

for state in states:
    # path here
    base_dir = Path(f"E:/study/meteoroData/{state}")

    # regular expression to read name of file
    pattern = re.compile(r"^(.+)-(\d{6})\.csv$")

    for csv_path in base_dir.glob("*.csv"):
        m = pattern.match(csv_path.name)
        if not m:
            # skip the one does not math any
            print(f"skip does not match format：{csv_path.name}")
            continue

        loc_name, yyyymm = m.groups()
        # 1) make file base on location name
        target_dir = base_dir / loc_name
        target_dir.mkdir(exist_ok=True)

        # 2) move csv to file
        dest_path = target_dir / csv_path.name
        shutil.move(str(csv_path), str(dest_path))
        print(f"moved: {csv_path.name} -> {loc_name}/")

moved: alice_springs_airport-202205.csv -> alice_springs_airport/
moved: alice_springs_airport-202206.csv -> alice_springs_airport/
moved: alice_springs_airport-202207.csv -> alice_springs_airport/
moved: alice_springs_airport-202208.csv -> alice_springs_airport/
moved: alice_springs_airport-202209.csv -> alice_springs_airport/
moved: alice_springs_airport-202210.csv -> alice_springs_airport/
moved: alice_springs_airport-202211.csv -> alice_springs_airport/
moved: alice_springs_airport-202212.csv -> alice_springs_airport/
moved: alice_springs_airport-202301.csv -> alice_springs_airport/
moved: alice_springs_airport-202302.csv -> alice_springs_airport/
moved: alice_springs_airport-202303.csv -> alice_springs_airport/
moved: alice_springs_airport-202304.csv -> alice_springs_airport/
moved: alice_springs_airport-202305.csv -> alice_springs_airport/
moved: alice_springs_airport-202306.csv -> alice_springs_airport/
moved: alice_springs_airport-202307.csv -> alice_springs_airport/
moved: ali

moved: borroloola_airport-202410.csv -> borroloola_airport/
moved: borroloola_airport-202411.csv -> borroloola_airport/
moved: borroloola_airport-202412.csv -> borroloola_airport/
moved: borroloola_airport-202501.csv -> borroloola_airport/
moved: borroloola_airport-202502.csv -> borroloola_airport/
moved: borroloola_airport-202503.csv -> borroloola_airport/
moved: borroloola_airport-202504.csv -> borroloola_airport/
moved: bradshaw-202205.csv -> bradshaw/
moved: bradshaw-202206.csv -> bradshaw/
moved: bradshaw-202207.csv -> bradshaw/
moved: bradshaw-202208.csv -> bradshaw/
moved: bradshaw-202209.csv -> bradshaw/
moved: bradshaw-202210.csv -> bradshaw/
moved: bradshaw-202211.csv -> bradshaw/
moved: bradshaw-202212.csv -> bradshaw/
moved: bradshaw-202301.csv -> bradshaw/
moved: bradshaw-202302.csv -> bradshaw/
moved: bradshaw-202303.csv -> bradshaw/
moved: bradshaw-202304.csv -> bradshaw/
moved: bradshaw-202305.csv -> bradshaw/
moved: bradshaw-202306.csv -> bradshaw/
moved: bradshaw-2023

moved: bulman-202208.csv -> bulman/
moved: bulman-202209.csv -> bulman/
moved: bulman-202210.csv -> bulman/
moved: bulman-202211.csv -> bulman/
moved: bulman-202212.csv -> bulman/
moved: bulman-202301.csv -> bulman/
moved: bulman-202302.csv -> bulman/
moved: bulman-202303.csv -> bulman/
moved: bulman-202304.csv -> bulman/
moved: bulman-202305.csv -> bulman/
moved: bulman-202306.csv -> bulman/
moved: bulman-202307.csv -> bulman/
moved: bulman-202308.csv -> bulman/
moved: bulman-202309.csv -> bulman/
moved: bulman-202310.csv -> bulman/
moved: bulman-202311.csv -> bulman/
moved: bulman-202312.csv -> bulman/
moved: bulman-202401.csv -> bulman/
moved: bulman-202402.csv -> bulman/
moved: bulman-202403.csv -> bulman/
moved: bulman-202404.csv -> bulman/
moved: bulman-202405.csv -> bulman/
moved: bulman-202406.csv -> bulman/
moved: bulman-202407.csv -> bulman/
moved: bulman-202408.csv -> bulman/
moved: bulman-202409.csv -> bulman/
moved: bulman-202410.csv -> bulman/
moved: bulman-202411.csv -> 